# 四大模块统一测试（A/B/C/D）

说明：本 notebook 按模块分成 4 个运行 block（A/B/C/D），每个 block 单独输出该模块的测试结果（通过/跳过/失败列表）。

默认使用 `../build` 目录；如需重新编译，把 `BUILD_FIRST = True`，先运行第一个 code cell。

In [4]:
from __future__ import annotations

import os
import subprocess
from pathlib import Path

ROOT = (Path("..") if Path.cwd().name == "notebooks" else Path(".")).resolve()
BUILD_DIR = (ROOT / "build").resolve()

# 基本参数
TIMEOUT_S = 30
BUILD_FIRST = False

# 每个模块内部：失败后是否继续跑完该模块
CONTINUE_ON_FAIL = False

# 模块用例分组
TESTS_A = [
    "test_eventloop",
    "test_select_poller",
    "test_tcpserver",
    "test_tcpclient",
    "test_connector_retry",
    "test_idle_cleanup",
    "test_udp_proxy",
    "test_l4_tunnel",
    "test_conn_limit",
    "test_conn_limit_user_service",
    "test_allocator",
    "test_buddy_allocator",
    "test_memory_stats",
    "test_hugepage_slab",
    "test_multithread",
]

TESTS_B = [
    "test_balancer",
    "test_leastconn",
    "test_leastqueue",
    "test_rtw",
    "test_gpu_balancer",
    "test_balancer_health",
    "test_http_health_checker",
    "test_ai_service_check",
    "test_script_health_checker",
    "test_warmup",
    "test_backend_manager",
    "test_backend_conn_pool",
    "test_model_affinity",
    "test_service_discovery",
    "test_dynamic_register",
    "test_passive_failover",
]

TESTS_C = [
    "test_http",
    "test_httpserver",
    "test_http_keepalive_chunked",
    "test_http2_basic",
    "test_grpc_h2c",
    "test_websocket_upgrade",
    "test_protocol_conversion",
    "test_compression_conversion",
    "test_rewrite_rules",
    "test_traffic_mirror",
    "test_api_aggregate",
    "test_cache_memcached",
    "test_cache_redis",
    "test_http_batching",
    "test_batch_split",
    "test_streaming_response",
    "test_cookie",
    "test_model_version_routing",
]

TESTS_D = [
    "test_stats_json",
    "test_stats_backends",
    "test_history",
    "test_dashboard",
    "test_diagnostics",
    "test_config_as_code",
    "test_config_web",
    "test_alert_webhook",
    "test_alert_channels",
    "test_alert_anomaly",
    "test_token_bucket",
    "test_per_ip_rate_limit",
    "test_per_path_rate_limit",
    "test_access_control",
    "test_audit_logger",
    "test_ddos_accept_limit",
    "test_congestion_control",
    "test_priority_queue",
    "test_fair_queue",
    "test_edf",
    "test_tls_acme",
    "test_plugin_manager",
    "test_plugin_http",
]

TEST_GROUPS: dict[str, list[str]] = {"A": TESTS_A, "B": TESTS_B, "C": TESTS_C, "D": TESTS_D}
results: dict[str, dict] = {}


def sh(cmd: list[str], *, cwd: Path = ROOT, timeout: int | None = None) -> None:
    print("$", " ".join(cmd))
    subprocess.run(cmd, cwd=str(cwd), check=True, timeout=timeout)


def maybe_build() -> None:
    BUILD_DIR.mkdir(parents=True, exist_ok=True)
    cache = BUILD_DIR / "CMakeCache.txt"
    if not cache.exists():
        sh(["cmake", "-S", str(ROOT), "-B", str(BUILD_DIR)])
    jobs = str(max(1, os.cpu_count() or 4))
    sh(["cmake", "--build", str(BUILD_DIR), "-j", jobs])


def _format_rc(rc: int) -> str:
    if rc < 0:
        return f"signal={-rc}"
    return f"exit={rc}"


def run_module(m: str) -> dict:
    tests = TEST_GROUPS.get(m, [])
    ran: list[str] = []
    skipped: list[str] = []
    failures: list[tuple[str, str]] = []

    print("\n" + "=" * 60)
    print(f"MODULE {m}")
    print("=" * 60)

    for t in tests:
        p = BUILD_DIR / t
        if not p.exists():
            print(f"SKIP {t} (missing: {p})")
            skipped.append(t)
            continue

        print(f"RUN {t}")
        try:
            sh([str(p)], cwd=ROOT, timeout=TIMEOUT_S)
            ran.append(t)
        except subprocess.TimeoutExpired:
            print(f"FAIL {t} timeout={TIMEOUT_S}s")
            failures.append((t, f"timeout={TIMEOUT_S}s"))
            if not CONTINUE_ON_FAIL:
                break
        except subprocess.CalledProcessError as e:
            print(f"FAIL {t} {_format_rc(e.returncode)}")
            failures.append((t, _format_rc(e.returncode)))
            if not CONTINUE_ON_FAIL:
                break

    summary = {
        "module": m,
        "ran": ran,
        "skipped": skipped,
        "failures": failures,
        "ok": len(failures) == 0,
    }

    print("\n--- SUMMARY ---")
    print("OK" if summary["ok"] else "FAIL")
    print("ran:", len(ran), "skipped:", len(skipped), "failed:", len(failures))
    if failures:
        print("failed list:", ", ".join([f"{n}({r})" for n, r in failures]))
    return summary


if BUILD_FIRST:
    maybe_build()


In [5]:
# Block A
results["A"] = run_module("A")



MODULE A
RUN test_eventloop
$ /home/huazi/Workspace/Projects/linux大作业/build/test_eventloop
[2026-01-14 19:19:41.383] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_eventloop.cpp:41] Starting EventLoop test
[2026-01-14 19:19:41.383] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/src/network/EpollPoller.cpp:147] Using EpollPoller
[2026-01-14 19:19:41.383] [DEBUG] [/home/huazi/Workspace/Projects/linux大作业/src/network/EventLoop.cpp:39] EventLoop created 0x7fff1db86f70 in thread 140018339403648
[2026-01-14 19:19:41.383] [DEBUG] [/home/huazi/Workspace/Projects/linux大作业/src/network/EpollPoller.cpp:68] fd = 4 events = 3 index = -1
[2026-01-14 19:19:41.383] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/src/network/EventLoop.cpp:61] EventLoop 0x7fff1db86f70 start looping
[2026-01-14 19:19:42.384] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_eventloop.cpp:48] Quitting main loop from thread
[2026-01-14 19:19:42.384] [DEBUG] [/home/huazi/Workspace/Projects

In [6]:
# Block B
results["B"] = run_module("B")



MODULE B
RUN test_balancer
$ /home/huazi/Workspace/Projects/linux大作业/build/test_balancer
[2026-01-14 19:19:59.216] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:11] Starting ConsistentHashBalancer test
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:28] Distribution Results (10000 requests):
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:30]   ServerA: 5317 (53.17%)
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:30]   ServerB: 1443 (14.43%)
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:30]   ServerC: 3240 (32.4%)
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_balancer.cpp:38] Consistency check: PASS
[2026-01-14 19:19:59.219] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/te

In [ ]:
# Block C
results["C"] = run_module("C")



MODULE C
RUN test_http
$ /home/huazi/Workspace/Projects/linux大作业/build/test_http
[2026-01-14 19:20:09.563] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/src/common/MemoryPool.cpp:53] MemoryPool buddy enabled: min_kb=128 arena_kb=8192 keep_arenas=1 max_arenas=8
[2026-01-14 19:20:09.563] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_http.cpp:41] Parse Request PASS
[2026-01-14 19:20:09.563] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_http.cpp:67] Parse Content-Length Body PASS
[2026-01-14 19:20:09.563] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_http.cpp:90] Parse Chunked Body PASS
[2026-01-14 19:20:09.563] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/tests/unit/test_http.cpp:114] Response Gen PASS
RUN test_httpserver
$ /home/huazi/Workspace/Projects/linux大作业/build/test_httpserver
[2026-01-14 19:20:09.581] [INFO ] [/home/huazi/Workspace/Projects/linux大作业/src/network/EpollPoller.cpp:147] Using EpollPoller
[2026-01-14 19:2

In [ ]:
# Block D
results["D"] = run_module("D")
